Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [x] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [x] Do one-hot encoding of categorical features.
- [x] Do feature selection with `SelectKBest`.
- [x] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [x] Fit a ridge regression model with multiple features.
- [x] Get mean absolute error for the test set.
- [x] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [x] Add your own stretch goal(s) ! - Tried the CatBoost model.
- [x] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [x] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [x] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [x] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [4]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

### Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [5]:
df.head().T

,0,1,2,3,4
BOROUGH,1,1,1,1,1
NEIGHBORHOOD,OTHER,OTHER,OTHER,OTHER,UPPER EAST SIDE (59-79)
BUILDING_CLASS_CATEGORY,13 CONDOS - ELEVATOR APARTMENTS,21 OFFICE BUILDINGS,21 OFFICE BUILDINGS,13 CONDOS - ELEVATOR APARTMENTS,15 CONDOS - 2-10 UNIT RESIDENTIAL
TAX_CLASS_AT_PRESENT,2,4,4,2,2C
BLOCK,716,812,839,592,1379
LOT,1246,68,69,1041,1402
EASE-MENT,NaN,NaN,NaN,NaN,NaN
BUILDING_CLASS_AT_PRESENT,R4,O5,O5,R4,R1
ADDRESS,"447 WEST 18TH STREET, PH12A",144 WEST 37TH STREET,40 WEST 38TH STREET,"1 SHERIDAN SQUARE, 8C","20 EAST 65TH STREET, B"
APARTMENT_NUMBER,PH12A,NaN,NaN,8C,B


In [6]:
df.groupby('SALE_DATE')['SALE_PRICE'].mean()

# Starts from 01/01/2019 - 04/30/2019

SALE_DATE
01/01/2019    1.153846e+04
01/02/2019    5.415812e+05
01/03/2019    9.637578e+05
01/04/2019    6.682856e+05
01/05/2019    5.736461e+04
01/06/2019    7.525000e+04
01/07/2019    8.507815e+05
01/08/2019    4.847869e+06
01/09/2019    2.159193e+06
01/10/2019    2.184470e+06
01/11/2019    1.072817e+06
01/12/2019    1.148185e+03
01/13/2019    1.000000e+04
01/14/2019    6.623459e+05
01/15/2019    9.232334e+05
01/16/2019    1.096404e+06
01/17/2019    1.058700e+06
01/18/2019    7.017070e+05
01/19/2019    6.068273e+04
01/20/2019    0.000000e+00
01/21/2019    1.645138e+05
01/22/2019    2.188749e+06
01/23/2019    2.928732e+06
01/24/2019    6.718692e+05
01/25/2019    1.352832e+06
01/26/2019    0.000000e+00
01/27/2019    2.500000e+00
01/28/2019    7.713877e+05
01/29/2019    1.243344e+06
01/30/2019    9.035125e+05
                  ...     
04/01/2019    4.453597e+06
04/02/2019    9.932023e+05
04/03/2019    1.008916e+06
04/04/2019    2.001412e+06
04/05/2019    2.929294e+06
04/06/2019    0.00

In [7]:
df2 = df
print(df2.shape)

df2.drop(columns=['EASE-MENT', 'APARTMENT_NUMBER'], inplace=True, errors='ignore') # >90% NaN values

df2['SALE_DATE'] = pd.to_datetime(df2['SALE_DATE'], infer_datetime_format = True)

cutoff = pd.to_datetime('04/01/2019')

train = df2[df2.SALE_DATE < cutoff]
test = df2[df2.SALE_DATE >= cutoff]
print(train.shape, test.shape)
train.dropna(inplace=True)
test.dropna(inplace=True)
print(train.shape, test.shape)

(23040, 21)
(18167, 19) (4873, 19)
(18134, 19) (4819, 19)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Do one-hot encoding of categorical features. - CatBoost

In [8]:
train.select_dtypes(exclude='number').describe().T

# Disregard features with a high cardinality

,count,unique,top,freq,first,last
BOROUGH,18134,5,4,5866,NaN,NaN
NEIGHBORHOOD,18134,11,OTHER,15005,NaN,NaN
BUILDING_CLASS_CATEGORY,18134,43,01 ONE FAMILY DWELLINGS,4094,NaN,NaN
TAX_CLASS_AT_PRESENT,18134,10,1,8911,NaN,NaN
BUILDING_CLASS_AT_PRESENT,18134,140,D4,2640,NaN,NaN
ADDRESS,18134,17893,N/A ROCKAWAY BOULEVARD,5,NaN,NaN
LAND_SQUARE_FEET,18134,3207,0,5780,NaN,NaN
BUILDING_CLASS_AT_TIME_OF_SALE,18134,140,D4,2640,NaN,NaN
SALE_DATE,18134,90,2019-01-24 00:00:00,480,2019-01-01 00:00:00,2019-03-31 00:00:00


In [9]:
features = ['BOROUGH',
 'NEIGHBORHOOD',
 'BUILDING_CLASS_CATEGORY',
 'TAX_CLASS_AT_PRESENT',
 'BUILDING_CLASS_AT_PRESENT',
 'BLOCK',
 'LOT',
 'EASE-MENT',
 'ZIP_CODE',
 'RESIDENTIAL_UNITS',
 'COMMERCIAL_UNITS',
 'TOTAL_UNITS',
 'GROSS_SQUARE_FEET',
 'YEAR_BUILT',
 'TAX_CLASS_AT_TIME_OF_SALE']

In [10]:
# https://towardsdatascience.com/benchmarking-categorical-encoders-9c322bd77ee8
# https://github.com/DenisVorotyntsev/CategoricalEncodingBenchmark/blob/master/show_results.ipynb
# https://contrib.scikit-learn.org/categorical-encoding/catboost.html

encode_list = ['BOROUGH', 'NEIGHBORHOOD', 
               'BUILDING_CLASS_CATEGORY',
              'TAX_CLASS_AT_PRESENT',
              'BUILDING_CLASS_AT_PRESENT']

num_values = train.select_dtypes(include='number').columns.values.tolist()

features = ['BOROUGH',
 'NEIGHBORHOOD',
 'BUILDING_CLASS_CATEGORY',
 'TAX_CLASS_AT_PRESENT',
 'BUILDING_CLASS_AT_PRESENT',
 'BLOCK',
 'LOT',
 #'EASE-MENT', # NaN values
 'ZIP_CODE',
 'RESIDENTIAL_UNITS',
 'COMMERCIAL_UNITS',
 'TOTAL_UNITS',
 'GROSS_SQUARE_FEET',
 'YEAR_BUILT',
 'TAX_CLASS_AT_TIME_OF_SALE']

target = 'SALE_PRICE'

import category_encoders as ce
from category_encoders.cat_boost import CatBoostEncoder

# cb_encoder = ce.CatBoostEncoder(cols=[encode_list])
# I couldn't figure out why the cols parameter wouldn't work

cb_encoder = ce.CatBoostEncoder()
cb_train_encoded = cb_encoder.fit_transform(train[features], train[target])
cb_test_encoded = cb_encoder.transform(test[features], test[target])
cb_train_encoded.head().T

,0,1,2,3,4
BOROUGH,1.217694e+06,6.088469e+05,405897.942465,304423.456849,2.435388e+05
NEIGHBORHOOD,1.217694e+06,6.088469e+05,405897.942465,304423.456849,1.217694e+06
BUILDING_CLASS_CATEGORY,1.217694e+06,1.217694e+06,608846.913698,608846.913698,1.217694e+06
TAX_CLASS_AT_PRESENT,1.217694e+06,1.217694e+06,608846.913698,608846.913698,1.217694e+06
BUILDING_CLASS_AT_PRESENT,1.217694e+06,1.217694e+06,608846.913698,608846.913698,1.217694e+06
BLOCK,7.160000e+02,8.120000e+02,839.000000,592.000000,1.379000e+03
LOT,1.246000e+03,6.800000e+01,69.000000,1041.000000,1.402000e+03
ZIP_CODE,1.001100e+04,1.001800e+04,10018.000000,10014.000000,1.006500e+04
RESIDENTIAL_UNITS,1.000000e+00,0.000000e+00,0.000000,1.000000,1.000000e+00
COMMERCIAL_UNITS,0.000000e+00,6.000000e+00,7.000000,0.000000,0.000000e+00


In [11]:
X_train = cb_train_encoded.dropna()
X_test = cb_test_encoded.dropna()
y_train = train[target]
y_test = test[target]

In [12]:
# Here I dropped rows from the target arrays in order to 
# match the length of the feature datasets

In [13]:
# print(X_train.shape[0], y_train.shape[0])
# y_train = y_train[:-5]
# print(X_train.shape[0], y_train.shape[0])

In [14]:
# print(X_test.shape[0], y_test.shape[0])
# y_test = y_test[:-30]
# print(X_test.shape[0], y_test.shape[0])

In [15]:
# Before encoding:

train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,R4,"447 WEST 18TH STREET, PH12A",10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,2019-01-01
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,O5,144 WEST 37TH STREET,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,2019-01-01
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,O5,40 WEST 38TH STREET,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,2019-01-01
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,R4,"1 SHERIDAN SQUARE, 8C",10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,2019-01-01
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,R1,"20 EAST 65TH STREET, B",10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,2019-01-01


In [16]:
# After encoding:

X_train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
0,1.217694e+06,1.217694e+06,1.217694e+06,1.217694e+06,1.217694e+06,716,1246,10011.0,1.0,0.0,1.0,1979.0,2007.0,2
1,6.088469e+05,6.088469e+05,1.217694e+06,1.217694e+06,1.217694e+06,812,68,10018.0,0.0,6.0,6.0,15435.0,1920.0,4
2,4.058979e+05,4.058979e+05,6.088469e+05,6.088469e+05,6.088469e+05,839,69,10018.0,0.0,7.0,7.0,11332.0,1930.0,4
3,3.044235e+05,3.044235e+05,6.088469e+05,6.088469e+05,6.088469e+05,592,1041,10014.0,1.0,0.0,1.0,500.0,0.0,2
4,2.435388e+05,1.217694e+06,1.217694e+06,1.217694e+06,1.217694e+06,1379,1402,10065.0,1.0,0.0,1.0,6406.0,0.0,2


In [17]:
%%time

from catboost import CatBoost, CatBoostClassifier

model = CatBoostClassifier(one_hot_max_size=200)
model.fit(train['BOROUGH'][0:100], train[target][0:100])

# Out of 'encode_list', only 'BOROUGH' contains compatible dtypes to use
# this model. [0:50] slicing was used because the entire train set took
# a projected time of ~3 DAYS to execute

0:	learn: 2.8953645	total: 55.7ms	remaining: 55.7s
1:	learn: 2.7306529	total: 61.8ms	remaining: 30.8s
2:	learn: 2.6298527	total: 68.4ms	remaining: 22.7s
3:	learn: 2.5489661	total: 75.3ms	remaining: 18.7s
4:	learn: 2.4466445	total: 80.8ms	remaining: 16.1s
5:	learn: 2.3654613	total: 85.6ms	remaining: 14.2s
6:	learn: 2.2818879	total: 89.5ms	remaining: 12.7s
7:	learn: 2.2129418	total: 93.8ms	remaining: 11.6s
8:	learn: 2.1510889	total: 97.5ms	remaining: 10.7s
9:	learn: 2.0960102	total: 99.3ms	remaining: 9.83s
10:	learn: 2.0422827	total: 101ms	remaining: 9.08s
11:	learn: 1.9980399	total: 105ms	remaining: 8.68s
12:	learn: 1.9444917	total: 112ms	remaining: 8.51s
13:	learn: 1.9014649	total: 120ms	remaining: 8.43s
14:	learn: 1.8595009	total: 124ms	remaining: 8.11s
15:	learn: 1.8176245	total: 128ms	remaining: 7.85s
16:	learn: 1.7839776	total: 132ms	remaining: 7.61s
17:	learn: 1.7499245	total: 134ms	remaining: 7.33s
18:	learn: 1.7178058	total: 137ms	remaining: 7.05s
19:	learn: 1.6900907	total: 142

214:	learn: 0.9985382	total: 631ms	remaining: 2.3s
215:	learn: 0.9979191	total: 633ms	remaining: 2.3s
216:	learn: 0.9974112	total: 635ms	remaining: 2.29s
217:	learn: 0.9968010	total: 639ms	remaining: 2.29s
218:	learn: 0.9962253	total: 646ms	remaining: 2.3s
219:	learn: 0.9957642	total: 652ms	remaining: 2.31s
220:	learn: 0.9951966	total: 658ms	remaining: 2.32s
221:	learn: 0.9946051	total: 661ms	remaining: 2.32s
222:	learn: 0.9940186	total: 663ms	remaining: 2.31s
223:	learn: 0.9934371	total: 666ms	remaining: 2.31s
224:	learn: 0.9928871	total: 670ms	remaining: 2.31s
225:	learn: 0.9923410	total: 672ms	remaining: 2.3s
226:	learn: 0.9919068	total: 675ms	remaining: 2.3s
227:	learn: 0.9913424	total: 677ms	remaining: 2.29s
228:	learn: 0.9907827	total: 678ms	remaining: 2.28s
229:	learn: 0.9902276	total: 680ms	remaining: 2.28s
230:	learn: 0.9896770	total: 683ms	remaining: 2.27s
231:	learn: 0.9891546	total: 684ms	remaining: 2.27s
232:	learn: 0.9887714	total: 688ms	remaining: 2.26s
233:	learn: 0.988

428:	learn: 0.9292491	total: 1.2s	remaining: 1.59s
429:	learn: 0.9290752	total: 1.2s	remaining: 1.6s
430:	learn: 0.9289021	total: 1.21s	remaining: 1.6s
431:	learn: 0.9287298	total: 1.22s	remaining: 1.6s
432:	learn: 0.9285584	total: 1.22s	remaining: 1.6s
433:	learn: 0.9283877	total: 1.22s	remaining: 1.59s
434:	learn: 0.9282179	total: 1.23s	remaining: 1.59s
435:	learn: 0.9280450	total: 1.23s	remaining: 1.59s
436:	learn: 0.9278767	total: 1.23s	remaining: 1.58s
437:	learn: 0.9277092	total: 1.23s	remaining: 1.58s
438:	learn: 0.9275387	total: 1.23s	remaining: 1.58s
439:	learn: 0.9273727	total: 1.24s	remaining: 1.57s
440:	learn: 0.9272038	total: 1.24s	remaining: 1.57s
441:	learn: 0.9270357	total: 1.24s	remaining: 1.56s
442:	learn: 0.9268721	total: 1.24s	remaining: 1.56s
443:	learn: 0.9267055	total: 1.24s	remaining: 1.56s
444:	learn: 0.9265434	total: 1.25s	remaining: 1.55s
445:	learn: 0.9263784	total: 1.25s	remaining: 1.55s
446:	learn: 0.9262177	total: 1.25s	remaining: 1.55s
447:	learn: 0.9260

587:	learn: 0.9091700	total: 1.58s	remaining: 1.11s
588:	learn: 0.9090786	total: 1.59s	remaining: 1.11s
589:	learn: 0.9089875	total: 1.59s	remaining: 1.11s
590:	learn: 0.9088994	total: 1.6s	remaining: 1.1s
591:	learn: 0.9088116	total: 1.6s	remaining: 1.1s
592:	learn: 0.9087216	total: 1.6s	remaining: 1.1s
593:	learn: 0.9086344	total: 1.61s	remaining: 1.1s
594:	learn: 0.9085476	total: 1.61s	remaining: 1.09s
595:	learn: 0.9084611	total: 1.61s	remaining: 1.09s
596:	learn: 0.9083750	total: 1.61s	remaining: 1.09s
597:	learn: 0.9082891	total: 1.61s	remaining: 1.08s
598:	learn: 0.9082036	total: 1.62s	remaining: 1.08s
599:	learn: 0.9081184	total: 1.62s	remaining: 1.08s
600:	learn: 0.9080310	total: 1.62s	remaining: 1.08s
601:	learn: 0.9079464	total: 1.62s	remaining: 1.07s
602:	learn: 0.9078621	total: 1.63s	remaining: 1.07s
603:	learn: 0.9077782	total: 1.63s	remaining: 1.07s
604:	learn: 0.9076920	total: 1.63s	remaining: 1.06s
605:	learn: 0.9076086	total: 1.63s	remaining: 1.06s
606:	learn: 0.90752

755:	learn: 0.8977503	total: 1.96s	remaining: 634ms
756:	learn: 0.8976981	total: 1.97s	remaining: 632ms
757:	learn: 0.8976479	total: 1.98s	remaining: 631ms
758:	learn: 0.8975978	total: 1.98s	remaining: 629ms
759:	learn: 0.8975478	total: 1.98s	remaining: 626ms
760:	learn: 0.8974980	total: 1.98s	remaining: 623ms
761:	learn: 0.8974484	total: 1.99s	remaining: 620ms
762:	learn: 0.8973989	total: 1.99s	remaining: 618ms
763:	learn: 0.8973496	total: 1.99s	remaining: 615ms
764:	learn: 0.8973004	total: 1.99s	remaining: 612ms
765:	learn: 0.8972513	total: 2s	remaining: 610ms
766:	learn: 0.8972024	total: 2s	remaining: 607ms
767:	learn: 0.8971519	total: 2s	remaining: 604ms
768:	learn: 0.8971015	total: 2s	remaining: 601ms
769:	learn: 0.8970530	total: 2s	remaining: 598ms
770:	learn: 0.8970047	total: 2s	remaining: 595ms
771:	learn: 0.8969547	total: 2.01s	remaining: 593ms
772:	learn: 0.8969067	total: 2.01s	remaining: 590ms
773:	learn: 0.8968571	total: 2.01s	remaining: 587ms
774:	learn: 0.8968093	total: 2

920:	learn: 0.8910027	total: 2.35s	remaining: 201ms
921:	learn: 0.8909705	total: 2.35s	remaining: 199ms
922:	learn: 0.8909373	total: 2.36s	remaining: 197ms
923:	learn: 0.8909053	total: 2.36s	remaining: 194ms
924:	learn: 0.8908723	total: 2.37s	remaining: 192ms
925:	learn: 0.8908393	total: 2.37s	remaining: 189ms
926:	learn: 0.8908076	total: 2.37s	remaining: 187ms
927:	learn: 0.8907759	total: 2.37s	remaining: 184ms
928:	learn: 0.8907432	total: 2.38s	remaining: 182ms
929:	learn: 0.8907105	total: 2.38s	remaining: 179ms
930:	learn: 0.8906791	total: 2.38s	remaining: 177ms
931:	learn: 0.8906466	total: 2.38s	remaining: 174ms
932:	learn: 0.8906154	total: 2.38s	remaining: 171ms
933:	learn: 0.8905830	total: 2.39s	remaining: 169ms
934:	learn: 0.8905519	total: 2.39s	remaining: 166ms
935:	learn: 0.8905209	total: 2.39s	remaining: 163ms
936:	learn: 0.8904888	total: 2.39s	remaining: 161ms
937:	learn: 0.8904568	total: 2.39s	remaining: 158ms
938:	learn: 0.8904260	total: 2.4s	remaining: 156ms
939:	learn: 0

### Do one-hot encoding of categorical features. - OneHotEncoder

In [18]:
# https://contrib.scikit-learn.org/categorical-encoding/catboost.html

import category_encoders as ce

encode_list = ['BOROUGH', 'NEIGHBORHOOD', 
               'BUILDING_CLASS_CATEGORY',
              'TAX_CLASS_AT_PRESENT',
              'BUILDING_CLASS_AT_PRESENT']

target = 'SALE_PRICE'

encoder = ce.OneHotEncoder(use_cat_names=True)
train_encoded = encoder.fit_transform(train[encode_list])
train_encoded.head()

,BOROUGH_1,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_UPPER EAST SIDE (79-96),NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,...,BUILDING_CLASS_AT_PRESENT_T2,BUILDING_CLASS_AT_PRESENT_RA,BUILDING_CLASS_AT_PRESENT_G4,BUILDING_CLASS_AT_PRESENT_V2,BUILDING_CLASS_AT_PRESENT_P9,BUILDING_CLASS_AT_PRESENT_W4,BUILDING_CLASS_AT_PRESENT_W3,BUILDING_CLASS_AT_PRESENT_I7,BUILDING_CLASS_AT_PRESENT_H4,BUILDING_CLASS_AT_PRESENT_K6
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Do feature selection with `SelectKBest`.

In [19]:
from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func=f_regression, k=5) # k must be <= len(features)
train_selected = selector.fit_transform(X_train, y_train)
test_selected = selector.transform(X_test)
train_selected.shape

# kbest can only take float dtypes with no NaN values?

(18134, 5)

In [20]:
# TODO: Which features were selected?
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features Selected:')
for name in selected_names:
  print(name)

print('')
print('Features Not Selected:')
for name in unselected_names:
  print(name)

Features Selected:
BUILDING_CLASS_CATEGORY
BUILDING_CLASS_AT_PRESENT
COMMERCIAL_UNITS
TOTAL_UNITS
GROSS_SQUARE_FEET

Features Not Selected:
BOROUGH
NEIGHBORHOOD
TAX_CLASS_AT_PRESENT
BLOCK
LOT
ZIP_CODE
RESIDENTIAL_UNITS
YEAR_BUILT
TAX_CLASS_AT_TIME_OF_SALE


### Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Fit a ridge regression model with multiple features.

In [22]:
from sklearn.linear_model import Ridge

model = Ridge()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

### Get mean absolute error for the test set.

In [23]:
from sklearn.metrics import mean_absolute_error

ridge_mae = mean_absolute_error(y_test, y_pred)
print(f'${ridge_mae:,.2f}')

$3,207,156.47


### Stretch Goal - Add your own stretch goal(s) ! - Tried the CatBoost model.

In [24]:
import catboost

for k in range(1, len(X_train.columns) + 1):
  print(f'{k} features:')

  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  model = CatBoost()
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)

  mae = mean_absolute_error(y_test, y_pred)
  print(f'Test MAE: ${mae:,.0f} \n')

1 features:
0:	learn: 10880605.6819985	total: 7.09ms	remaining: 7.09s
1:	learn: 10833362.5066450	total: 12.4ms	remaining: 6.17s
2:	learn: 10788556.2375655	total: 15.4ms	remaining: 5.12s
3:	learn: 10746116.3105219	total: 18.4ms	remaining: 4.58s
4:	learn: 10705726.3368992	total: 21.1ms	remaining: 4.19s
5:	learn: 10667817.8804722	total: 27.6ms	remaining: 4.58s
6:	learn: 10633183.4426308	total: 32ms	remaining: 4.54s
7:	learn: 10599469.8028785	total: 35.2ms	remaining: 4.36s
8:	learn: 10567089.8916089	total: 38.2ms	remaining: 4.21s
9:	learn: 10536447.3662376	total: 41.7ms	remaining: 4.13s
10:	learn: 10507890.0956690	total: 44.5ms	remaining: 4s
11:	learn: 10480436.5796246	total: 48.1ms	remaining: 3.96s
12:	learn: 10454424.0496020	total: 50.8ms	remaining: 3.85s
13:	learn: 10429788.7094273	total: 53.6ms	remaining: 3.77s
14:	learn: 10406537.1057208	total: 60.1ms	remaining: 3.95s
15:	learn: 10384991.1804647	total: 64.4ms	remaining: 3.96s
16:	learn: 10364064.2261765	total: 67.4ms	remaining: 3.9s
1

147:	learn: 9988459.0411507	total: 489ms	remaining: 2.82s
148:	learn: 9988398.8010847	total: 496ms	remaining: 2.83s
149:	learn: 9988022.5099070	total: 502ms	remaining: 2.84s
150:	learn: 9987658.8237500	total: 508ms	remaining: 2.86s
151:	learn: 9987617.3765681	total: 513ms	remaining: 2.86s
152:	learn: 9987607.6077146	total: 520ms	remaining: 2.88s
153:	learn: 9987571.8200438	total: 527ms	remaining: 2.89s
154:	learn: 9987541.2372959	total: 532ms	remaining: 2.9s
155:	learn: 9987516.4081568	total: 538ms	remaining: 2.91s
156:	learn: 9987500.8181458	total: 541ms	remaining: 2.9s
157:	learn: 9987472.7568748	total: 543ms	remaining: 2.89s
158:	learn: 9987436.1677263	total: 550ms	remaining: 2.91s
159:	learn: 9987424.7864170	total: 557ms	remaining: 2.93s
160:	learn: 9987408.5422793	total: 564ms	remaining: 2.94s
161:	learn: 9987372.6978089	total: 570ms	remaining: 2.95s
162:	learn: 9987354.2962597	total: 575ms	remaining: 2.95s
163:	learn: 9987342.4094739	total: 580ms	remaining: 2.96s
164:	learn: 9987

291:	learn: 9983258.7718707	total: 905ms	remaining: 2.19s
292:	learn: 9983258.2191706	total: 910ms	remaining: 2.2s
293:	learn: 9983216.1346112	total: 916ms	remaining: 2.2s
294:	learn: 9983208.8121499	total: 923ms	remaining: 2.21s
295:	learn: 9983208.1140858	total: 930ms	remaining: 2.21s
296:	learn: 9983167.7894605	total: 938ms	remaining: 2.22s
297:	learn: 9983163.5129503	total: 944ms	remaining: 2.22s
298:	learn: 9983163.1398661	total: 950ms	remaining: 2.23s
299:	learn: 9983159.7209725	total: 956ms	remaining: 2.23s
300:	learn: 9983157.7192513	total: 963ms	remaining: 2.23s
301:	learn: 9983130.0054180	total: 970ms	remaining: 2.24s
302:	learn: 9983121.7022461	total: 974ms	remaining: 2.24s
303:	learn: 9983046.0576592	total: 981ms	remaining: 2.25s
304:	learn: 9982988.0827943	total: 986ms	remaining: 2.25s
305:	learn: 9982977.4389497	total: 989ms	remaining: 2.24s
306:	learn: 9982938.1867196	total: 991ms	remaining: 2.24s
307:	learn: 9982936.1895117	total: 994ms	remaining: 2.23s
308:	learn: 9982

458:	learn: 9977582.9330140	total: 1.45s	remaining: 1.7s
459:	learn: 9977552.3903925	total: 1.45s	remaining: 1.7s
460:	learn: 9977536.6453293	total: 1.45s	remaining: 1.7s
461:	learn: 9977521.7822118	total: 1.46s	remaining: 1.7s
462:	learn: 9977487.2530532	total: 1.46s	remaining: 1.7s
463:	learn: 9977473.2370051	total: 1.47s	remaining: 1.7s
464:	learn: 9977453.8305388	total: 1.47s	remaining: 1.69s
465:	learn: 9977420.2365260	total: 1.47s	remaining: 1.69s
466:	learn: 9977390.1727218	total: 1.48s	remaining: 1.69s
467:	learn: 9977354.9935539	total: 1.48s	remaining: 1.68s
468:	learn: 9977342.0881342	total: 1.49s	remaining: 1.68s
469:	learn: 9977319.0616840	total: 1.49s	remaining: 1.68s
470:	learn: 9977287.6534871	total: 1.5s	remaining: 1.68s
471:	learn: 9977253.2526210	total: 1.5s	remaining: 1.68s
472:	learn: 9977213.0641699	total: 1.51s	remaining: 1.68s
473:	learn: 9977184.8102052	total: 1.51s	remaining: 1.68s
474:	learn: 9977156.5837289	total: 1.52s	remaining: 1.68s
475:	learn: 9977142.34

601:	learn: 9975324.2412751	total: 1.88s	remaining: 1.24s
602:	learn: 9975311.5517656	total: 1.88s	remaining: 1.24s
603:	learn: 9975302.3398733	total: 1.89s	remaining: 1.24s
604:	learn: 9975299.9024339	total: 1.89s	remaining: 1.24s
605:	learn: 9975294.0374399	total: 1.9s	remaining: 1.23s
606:	learn: 9975282.2846555	total: 1.9s	remaining: 1.23s
607:	learn: 9975276.4289577	total: 1.9s	remaining: 1.23s
608:	learn: 9975262.3455243	total: 1.9s	remaining: 1.22s
609:	learn: 9975256.9340630	total: 1.91s	remaining: 1.22s
610:	learn: 9975248.8353658	total: 1.91s	remaining: 1.22s
611:	learn: 9975238.2496199	total: 1.91s	remaining: 1.21s
612:	learn: 9975228.9184047	total: 1.91s	remaining: 1.21s
613:	learn: 9975220.1844847	total: 1.91s	remaining: 1.2s
614:	learn: 9975216.4946551	total: 1.92s	remaining: 1.2s
615:	learn: 9975206.9634301	total: 1.92s	remaining: 1.2s
616:	learn: 9975193.6101354	total: 1.92s	remaining: 1.19s
617:	learn: 9975180.9931764	total: 1.92s	remaining: 1.19s
618:	learn: 9975168.0

...

312:	learn: 3112665.1495146	total: 1.02s	remaining: 2.23s
313:	learn: 3110321.6647178	total: 1.02s	remaining: 2.23s
314:	learn: 3104887.8102713	total: 1.02s	remaining: 2.23s
315:	learn: 3097737.8747225	total: 1.03s	remaining: 2.22s
316:	learn: 3092962.9683519	total: 1.03s	remaining: 2.22s
317:	learn: 3082297.6096773	total: 1.03s	remaining: 2.21s
318:	learn: 3076613.3809070	total: 1.04s	remaining: 2.22s
319:	learn: 3071284.4161209	total: 1.04s	remaining: 2.22s
320:	learn: 3063847.7945386	total: 1.05s	remaining: 2.22s
321:	learn: 3058676.7404382	total: 1.06s	remaining: 2.22s
322:	learn: 3053627.5130194	total: 1.06s	remaining: 2.22s
323:	learn: 3046828.6088153	total: 1.06s	remaining: 2.22s
324:	learn: 3045922.6480900	total: 1.07s	remaining: 2.21s
325:	learn: 3041075.6349572	total: 1.07s	remaining: 2.21s
326:	learn: 3032869.3345173	total: 1.07s	remaining: 2.2s
327:	learn: 3024783.0450544	total: 1.07s	remaining: 2.2s
328:	learn: 3020712.2614171	total: 1.08s	remaining: 2.19s
329:	learn: 3014

494:	learn: 2352506.0227049	total: 1.6s	remaining: 1.63s
495:	learn: 2349996.9547831	total: 1.61s	remaining: 1.63s
496:	learn: 2344429.0597327	total: 1.62s	remaining: 1.64s
497:	learn: 2342841.8630280	total: 1.62s	remaining: 1.64s
498:	learn: 2339945.3587908	total: 1.63s	remaining: 1.63s
499:	learn: 2337946.4568399	total: 1.63s	remaining: 1.63s
500:	learn: 2334574.1296603	total: 1.63s	remaining: 1.63s
501:	learn: 2330347.0878532	total: 1.64s	remaining: 1.62s
502:	learn: 2328813.9739333	total: 1.64s	remaining: 1.62s
503:	learn: 2323569.6096261	total: 1.64s	remaining: 1.62s
504:	learn: 2322554.7302615	total: 1.65s	remaining: 1.61s
505:	learn: 2319192.1491341	total: 1.65s	remaining: 1.61s
506:	learn: 2314224.0584184	total: 1.65s	remaining: 1.61s
507:	learn: 2311793.6043852	total: 1.66s	remaining: 1.6s
508:	learn: 2309890.0093228	total: 1.66s	remaining: 1.6s
509:	learn: 2307007.1282814	total: 1.66s	remaining: 1.6s
510:	learn: 2305516.2393123	total: 1.66s	remaining: 1.59s
511:	learn: 230399

665:	learn: 2001730.7179281	total: 2.17s	remaining: 1.09s
666:	learn: 1999937.2224630	total: 2.17s	remaining: 1.09s
667:	learn: 1996549.9224607	total: 2.18s	remaining: 1.08s
668:	learn: 1995057.4852551	total: 2.19s	remaining: 1.08s
669:	learn: 1994109.5188665	total: 2.19s	remaining: 1.08s
670:	learn: 1993697.1661663	total: 2.2s	remaining: 1.08s
671:	learn: 1992875.0930050	total: 2.2s	remaining: 1.07s
672:	learn: 1991095.2340348	total: 2.2s	remaining: 1.07s
673:	learn: 1989589.9405027	total: 2.21s	remaining: 1.07s
674:	learn: 1987312.9554855	total: 2.22s	remaining: 1.07s
675:	learn: 1984737.6298254	total: 2.22s	remaining: 1.07s
676:	learn: 1983048.7184018	total: 2.23s	remaining: 1.06s
677:	learn: 1982236.8569036	total: 2.24s	remaining: 1.06s
678:	learn: 1980863.4129007	total: 2.24s	remaining: 1.06s
679:	learn: 1980442.9933621	total: 2.25s	remaining: 1.06s
680:	learn: 1978379.4079122	total: 2.25s	remaining: 1.05s
681:	learn: 1976222.6104811	total: 2.25s	remaining: 1.05s
682:	learn: 19734

848:	learn: 1808377.6522674	total: 2.75s	remaining: 488ms
849:	learn: 1806722.7846008	total: 2.75s	remaining: 486ms
850:	learn: 1806289.0157414	total: 2.76s	remaining: 483ms
851:	learn: 1805587.8323956	total: 2.77s	remaining: 481ms
852:	learn: 1803713.2529922	total: 2.77s	remaining: 478ms
853:	learn: 1802507.2485951	total: 2.77s	remaining: 475ms
854:	learn: 1801823.4104889	total: 2.78s	remaining: 471ms
855:	learn: 1800422.2957937	total: 2.78s	remaining: 468ms
856:	learn: 1799366.2926735	total: 2.78s	remaining: 465ms
857:	learn: 1797887.9091279	total: 2.79s	remaining: 461ms
858:	learn: 1796537.6776861	total: 2.79s	remaining: 458ms
859:	learn: 1795889.0435560	total: 2.79s	remaining: 455ms
860:	learn: 1794887.7824923	total: 2.79s	remaining: 451ms
861:	learn: 1793904.6397013	total: 2.8s	remaining: 448ms
862:	learn: 1792945.3386965	total: 2.8s	remaining: 445ms
863:	learn: 1792341.0711808	total: 2.8s	remaining: 441ms
864:	learn: 1791110.2770969	total: 2.81s	remaining: 438ms
865:	learn: 17898

991:	learn: 1673235.2530696	total: 3.33s	remaining: 26.8ms
992:	learn: 1672451.8901511	total: 3.33s	remaining: 23.5ms
993:	learn: 1671996.9866856	total: 3.34s	remaining: 20.2ms
994:	learn: 1670471.8141364	total: 3.35s	remaining: 16.8ms
995:	learn: 1669123.4150265	total: 3.36s	remaining: 13.5ms
996:	learn: 1668677.4124682	total: 3.36s	remaining: 10.1ms
997:	learn: 1667251.5551494	total: 3.37s	remaining: 6.74ms
998:	learn: 1666799.0761706	total: 3.37s	remaining: 3.37ms
999:	learn: 1666775.7732551	total: 3.38s	remaining: 0us
Test MAE: $2,410,748 



In [25]:
# 11 features - Test MAE: $1,914,932
# This value is an improvement over LinearRegression and Ridge

### Stretch Goal - Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥

In [26]:
# You can try a range of values for k,
# then choose the model with the best score.
# If multiple models "tie" for the best score,
# choose the simplest model.
# You decide what counts as a tie!
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns) + 1):
  print(f'{k} features:')

  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  model = LinearRegression()
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)

  mae = mean_absolute_error(y_test, y_pred)
  print(f'Test MAE: ${mae:,.0f} \n')

1 features:
Test MAE: $2,053,411 

2 features:
Test MAE: $2,142,739 

3 features:
Test MAE: $2,146,373 

4 features:
Test MAE: $2,230,296 

5 features:
Test MAE: $2,156,507 

6 features:
Test MAE: $2,140,921 

7 features:
Test MAE: $2,153,292 

8 features:
Test MAE: $2,296,453 

9 features:
Test MAE: $2,966,121 

10 features:
Test MAE: $2,940,944 

11 features:
Test MAE: $2,935,988 

12 features:
Test MAE: $3,202,489 

13 features:
Test MAE: $3,203,440 

14 features:
Test MAE: $3,207,484 



### Stretch Goal - Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).

In [27]:
from sklearn.linear_model import RidgeCV

for k in range(1, len(X_train.columns) + 1):
  print(f'{k} features:')

  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  model = RidgeCV()
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)

  mae = mean_absolute_error(y_test, y_pred)
  print(f'Test MAE: ${mae:,.0f} \n')

1 features:
Test MAE: $2,053,743 

2 features:
Test MAE: $23,673,441 

3 features:
Test MAE: $13,797,735 

4 features:
Test MAE: $27,022,762 

5 features:
Test MAE: $60,997,936 

6 features:
Test MAE: $11,317,929 

7 features:
Test MAE: $76,010,985 

8 features:
Test MAE: $48,071,244 

9 features:
Test MAE: $66,474,943 

10 features:
Test MAE: $16,773,068 

11 features:
Test MAE: $51,185,788 

12 features:
Test MAE: $32,164,612 

13 features:
Test MAE: $53,075,693 

14 features:
Test MAE: $74,252,757 



### Stretch Goal - Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.

In [29]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

dat = sm.datasets.get_rdataset("Guerry", "HistData").data
results = smf.ols('Lottery ~ Literacy + np.log(Pop1831)', data=dat).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.333
Method:                 Least Squares   F-statistic:                     22.20
Date:                Thu, 07 Nov 2019   Prob (F-statistic):           1.90e-08
Time:                        13:10:52   Log-Likelihood:                -379.82
No. Observations:                  86   AIC:                             765.6
Df Residuals:                      83   BIC:                             773.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         246.4341     35.233     